In [ ]:
import os
import pandas as pd
import numpy as np
from collections import Counter

In [ ]:
######################################### str_information preparation  ######################################### 
current_file_path = os.getcwd()
# city = '500000' # '500000' incates Chongqing
# str_type_simple = 'bc' # bc rc wb
# building_function = 'r' # r b p i o x 
# epa = 0.2 #0.025 0.05 0.1 0.15 0.2 0.3 0.4 0.5
# rise_category = h1 # h1 h2 h3 h4 h5
def str_information_preparation(city,str_type_simple,building_function,epa,rise_category):
    ################ adjuest seismic design levels (for NA levels) by fortification intensity(g) and year
    path_str_inf = f"{current_file_path}\data_bulidings_raw"
    file_str_inf = os.path.join(path_str_inf,f"smp_{city}_{str_type_simple}_{building_function}_{epa}_{rise_category}.csv")
    df_str_inf = pd.read_csv(file_str_inf)
    
    ########### add num_story information
    df_str_inf['num_story'] = (df_str_inf['high'] / 3.0).round().astype(int)

    ########## add a structural type variable, str_type (for MDOFs), determined based on building height and construction era.
    num_str = df_str_inf.shape[0]
    str_type_list = []
    for str_i in range(num_str):
        year = df_str_inf.loc[str_i, 'year']
        high = df_str_inf.loc[str_i, 'high']
        if str_type_simple == 'rc':
            if year < 2001 and high > 50:
                str_type = 3 #reinforced concrete shear wall 3
            elif year > 2001 and high > 40:
                str_type = 2 #reinforced concrete frame-shear wall 2
            else: 
                str_type = 1 #reinforced concrete frame structure 1

        str_type_list.append(str_type)
    df_str_inf['str_type'] = str_type_list

    ############# Add seismic fortification intensity column to provide basis for subsequent structural mechanics performance parameter calibration
    def get_DI(row):
        value = float(epa)
        if value <= 0.05:
            return 6
        elif value <= 0.15+0.01:
            return 7
        elif value <= 0.3+0.01:
            return 8
        elif value <= 0.4:
            return 9
        else:
            return 9
    # Apply the function and create  new columns
    df_str_inf['DI'] = df_str_inf.apply(get_DI, axis=1)
    df_str_inf['epa'] = epa
    df_str_inf['tg'] = df_str_inf['td']
    
    return df_str_inf


In [ ]:
#########################################  get parameters files  #########################################
g = 9.8
path_GM = rf"{current_file_path}\Recommended Ground Motion" # Code for Collapse-Resistant Design of Building Structures
path_GM = path_GM.replace('\\', '/') 
# Read the basic ground motion information file into a DataFrame and assign column names.
df_GM_inf = {} 
df_GM_inf["Tg=0.3"] = pd.read_csv(f'{path_GM}/1_Site Class I_Tg0d30/GM_inf_list.txt', sep=' ', names=['iGMfile','iGMfact', 'dt', 'npts'], encoding='utf-8')
df_GM_inf["Tg=0.35"] = pd.read_csv(f'{path_GM}/2_Site Class I_Tg0d35/GM_inf_list.txt', sep=' ', names=['iGMfile','iGMfact', 'dt', 'npts'], encoding='utf-8')
df_GM_inf["Tg=0.4"] = pd.read_csv(f'{path_GM}/3_Site Class II_Tg0d40/GM_inf_list.txt', sep=' ', names=['iGMfile','iGMfact', 'dt', 'npts'], encoding='utf-8')
df_GM_inf["Tg=0.45"] = pd.read_csv(f'{path_GM}/4_Site Class II_Tg0d45/GM_inf_list.txt', sep=' ', names=['iGMfile','iGMfact', 'dt', 'npts'], encoding='utf-8')
df_GM_inf["Tg=0.5"] = pd.read_csv(f'{path_GM}/5_Site Class II_Tg0d50/GM_inf_list.txt', sep=' ', names=['iGMfile','iGMfact', 'dt', 'npts'], encoding='utf-8')

city_list = ['500000']
str_type_simple_list = ['rc'] # ['bc','rc','wd']
building_function_list = ['r','b','p','i','o','x'] # ['r','b','p','i','o','x'] Building function
epa_list = ['0.05','0.1','0.15','0.2','0.3','0.4','0.5'] #['0.025','0.05','0.1','0.15','0.2','0.3','0.4','0.5'] Design basic acceleration
rise_category_list = ['h5'] # ['h1','h2','h3','h4','h5']
sf_list = [0.025,0.05,0.075,0.1,0.2,0.3,0.5,0.8,1.0,1.5] # Amplification factor    0.025*g,0.05*g,0.075*g,0.1*g,0.2*g,0.3*g,0.5*g,0.8*g,1.0*g,1.5*g 

for city in  city_list:
    for str_type_simple in  str_type_simple_list:
        for building_function in  building_function_list:
            for epa in  epa_list:
                for rise_category in rise_category_list:
                    for sf in sf_list:
                        path_str_inf = f"{current_file_path}\data_bulidings_raw"
                        file_str_inf = os.path.join(path_str_inf,f"smp_{city}_{str_type_simple}_{building_function}_{epa}_{rise_category}.csv")
                        file_df_for_MDOFs = os.path.join(path_str_inf,f'city{{{city}}}_str_type_simple{{{str_type_simple}}}_building_function{{{building_function}}}_epa{{{epa}}}_rise_category{{{rise_category}}}_ScaleFactor{{{sf}}}.csv')

                        if os.path.exists(file_str_inf):
                            print("File exist:",file_str_inf)
                            # Extract structures for different cities, structural types (timber-brick, masonry-concrete, steel-concrete), seismic design levels, and height categories.
                            df_str_inf = str_information_preparation(city,str_type_simple,building_function,epa,rise_category)

                            # If the number of structures is 0, skip.
                            if df_str_inf.shape[0] == 0:
                                break
                            
                            # Uniform stratified random sampling: if the number of structures in a category is ≥ 200, select 200.
                            if df_str_inf.shape[0] >= 200:
                                shuffled_indices = np.random.permutation(df_str_inf.shape[0])
                                filtered_df = df_str_inf.iloc[shuffled_indices[:200],:]
                            else:
                                filtered_df = df_str_inf.copy()

                            # Create an empty DataFrame. Filter filtered_df again by Tg to obtain filtered_df_Tg_i, then combine it with the ground motions to form the parameter file. Concatenate iteratively.
                            df_for_MDOFs = pd.DataFrame()
                            for Tg_i in [0.3,0.35,0.4,0.45,0.5]: # 0.3,0.35,0.4,0.45,0.5
                                filtered_df_Tg_i = filtered_df[filtered_df['tg'] == Tg_i]
                                # If the number of rows in filtered_df_Tg_i is greater than 0, concatenate it.
                                if filtered_df_Tg_i.shape[0] > 0:
                                    df_numStory = pd.DataFrame(filtered_df_Tg_i['num_story'])
                                    df_numStory.columns = ['numStory']
                                    df_Fstorey_height = pd.DataFrame([3] * filtered_df_Tg_i.shape[0], columns=['Fstorey_height'])
                                    df_storey_height = pd.DataFrame([3] * filtered_df_Tg_i.shape[0], columns=['storey_height'])
                                    df_length = pd.DataFrame(filtered_df_Tg_i['chang'], columns=['chang'])
                                    df_length.columns=['length']
                                    df_breadth = pd.DataFrame(filtered_df_Tg_i['kuan'], columns=['kuan'])
                                    df_breadth.columns=['breadth']
                                    df_str_type = pd.DataFrame(filtered_df_Tg_i['str_type'], columns=['str_type'])
                                    df_site_class = pd.DataFrame([2] * filtered_df_Tg_i.shape[0] , columns=['site_class']) # Since it is not available, set it to 2 first; subsequent calculations directly use the characteristic period.
                                    df_seismic_class = pd.DataFrame([2] * filtered_df_Tg_i.shape[0] , columns=['seismic_class']) # Since it is not available, set it to 2 first; subsequent calculations directly use the characteristic period.
                                    df_DI = pd.DataFrame(filtered_df_Tg_i['DI'], columns=['DI'])
                                    df_DI.columns=['DI']
                                    df_DGA = pd.DataFrame(filtered_df_Tg_i['epa'], columns=['epa'])
                                    df_DGA.columns=['DGA']
                                    df_year = pd.DataFrame(filtered_df_Tg_i['year'], columns=['year'])
                                    df_tg = pd.DataFrame(filtered_df_Tg_i['tg'], columns=['tg']) # Site soil characteristic period
                                
                                    for GM_i in range(df_GM_inf[f'Tg={Tg_i}'].shape[0]):
                                        df_iGMfile =  pd.DataFrame([df_GM_inf[f'Tg={Tg_i}'].loc[GM_i,'iGMfile']] * filtered_df_Tg_i.shape[0], columns=['iGMfile'])
                                        df_iGMfact = pd.DataFrame([sf] * filtered_df_Tg_i.shape[0], columns=['iGMfact'])
                                        df_dt = pd.DataFrame([df_GM_inf[f'Tg={Tg_i}'].loc[GM_i,'dt']] * filtered_df_Tg_i.shape[0], columns=['dt'])
                                        df_npts = pd.DataFrame([df_GM_inf[f'Tg={Tg_i}'].loc[GM_i,'npts']] * filtered_df_Tg_i.shape[0], columns=['npts'])

                                        df_for_MDOFs_i = pd.concat([df_numStory.reset_index(drop=True), 
                                                    df_Fstorey_height.reset_index(drop=True), 
                                                    df_storey_height.reset_index(drop=True), 
                                                    df_length.reset_index(drop=True), 
                                                    df_breadth.reset_index(drop=True), 
                                                    df_str_type.reset_index(drop=True), 
                                                    df_site_class.reset_index(drop=True), 
                                                    df_seismic_class.reset_index(drop=True), 
                                                    df_DI.reset_index(drop=True), 
                                                    df_DGA.reset_index(drop=True), 
                                                    df_iGMfile.reset_index(drop=True), 
                                                    df_iGMfact.reset_index(drop=True), 
                                                    df_dt.reset_index(drop=True), 
                                                    df_npts.reset_index(drop=True), 
                                                    df_year.reset_index(drop=True), 
                                                    df_tg.reset_index(drop=True)], axis=1)

                                        if GM_i == 0:
                                            df_for_MDOFs_Tg_i = df_for_MDOFs_i
                                        else:
                                            df_for_MDOFs_Tg_i = pd.concat([df_for_MDOFs_Tg_i,df_for_MDOFs_i],axis=0,ignore_index=True)
                                    df_for_MDOFs = pd.concat([df_for_MDOFs,df_for_MDOFs_Tg_i],axis=0,ignore_index=True)
                            # After concatenating all Tg_i, save the result to a CSV file.
                            has_missing_values = df_for_MDOFs.isna().any().any()
                            if has_missing_values:
                                print("the df_for_MDOFs has missing values")
                            else:
                                path_str_inf = rf"{current_file_path}\data_bulidings_prepared\{city}"
                                if not os.path.exists(path_str_inf):
                                    print("Creating files:",path_str_inf)
                                    os.makedirs(path_str_inf)
                                df_for_MDOFs.index = df_for_MDOFs.index + 1     
                                df_for_MDOFs.to_csv(os.path.join(path_str_inf,f'city{{{city}}}_str_type_simple{{{str_type_simple}}}_building_function{{{building_function}}}_epa{{{epa}}}_rise_category{{{rise_category}}}_ScaleFactor{{{sf}}}.csv'))

                        else:
                            print("File does not exist:",file_str_inf)
    print(f"The parameter files of city {city} have been generated")